## Open a terminal window and run the following commands:
sudo bash
start-hadoop
cd /home/student/ROI/SparkProgram/Day3
./fixhive.sh


## Let's connect to the spark cluster

In [1]:
from pyspark.sql import SparkSession

# Create SparkSession 
spark = (SparkSession.builder 
          .master("local[1]") 
          .enableHiveSupport()
          .appName("jupyter") 
          .getOrCreate() 
        )
spark.sparkContext.setLogLevel("ERROR")
print('Done')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Done


## We can query an existing Hive table and bring it into a Spark DataFrame. First let's make sure we have a simple table in the default database.

In [ ]:
CREATE EXTERNAL TABLE regions(regionid int, regionname string)
ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' 
LINES TERMINATED BY '\n' 
STORED AS TEXTFILE
LOCATION '/regions';

LOAD DATA LOCAL INPATH '/class/datasets/northwind/CSV/regions' OVERWRITE INTO TABLE regions;


In [2]:
regions = spark.read.table('regions')
print(regions)
regions.show()

Loading class `com.mysql.jdbc.Driver'. This is deprecated. The new driver class is `com.mysql.cj.jdbc.Driver'. The driver is automatically registered via the SPI and manual loading of the driver class is generally unnecessary.


DataFrame[regionid: int, regionname: string]
+--------+----------+
|regionid|regionname|
+--------+----------+
|       1|   Eastern|
|       2|   Western|
|       3|  Northern|
|       4|  Southern|
+--------+----------+



## Or prefix the database name to use tables in northwind

In [5]:
territories = spark.read.table('northwind.territories')
print(territories)
territories.show()

DataFrame[territoryid: string, territoryname: string, regionid: int]
+-----------+-------------+--------+
|territoryid|territoryname|regionid|
+-----------+-------------+--------+
|      01581|     Westboro|       1|
|      01730|      Bedford|       1|
|      01833|    Georgetow|       1|
|      02116|       Boston|       1|
|      02139|    Cambridge|       1|
|      02184|    Braintree|       1|
|      02903|   Providence|       1|
|      03049|       Hollis|       3|
|      03801|   Portsmouth|       3|
|      06897|       Wilton|       1|
|      07960|   Morristown|       1|
|      08837|       Edison|       1|
|      10019|     New York|       1|
|      10038|     New York|       1|
|      11747|     Mellvile|       1|
|      14450|     Fairport|       1|
|      19428| Philadelphia|       3|
|      19713|       Neward|       1|
|      20852|    Rockville|       1|
|      27403|   Greensboro|       1|
+-----------+-------------+--------+
only showing top 20 rows



## This is a common helper function to make the output prettier

In [10]:
def display(df, limit = 10):
    from IPython.display import display    
    display(df.limit(limit).toPandas())


In [12]:
display(territories)

,territoryid,territoryname,regionid
0,01581,Westboro,1
1,01730,Bedford,1
2,01833,Georgetow,1
3,02116,Boston,1
4,02139,Cambridge,1
5,02184,Braintree,1
6,02903,Providence,1
7,03049,Hollis,3
8,03801,Portsmouth,3
9,06897,Wilton,1


### We can install this magic add in which lets us call SparkSQL directly

In [6]:
# pip3 install sparksql-magic
%load_ext sparksql_magic

In [7]:
%%sparksql
select * from northwind.regions


regionid,regionname
null,RegionName
1,Eastern
2,Western
3,Northern
4,Southern
5,Europe
6,Asia
7,
8,
9,null


### Can also capture the results to a variable and use it later

In [8]:
%%sparksql regions1
select regionname, regionid from regions where regionid < 4


capture dataframe to local variable `regions1`


regionname,regionid
Eastern,1
Western,2
Northern,3


## It's basically the same as this

In [ ]:
regions2 = spark.sql('select regionname, regionid from regions where regionid < 4')

## Once we have a reference to a DataFrame we could use dot methods on it to further process it

In [13]:
sort_field = 'regionname'
display(regions1.orderBy(sort_field))

,regionname,regionid
0,Eastern,1
1,Northern,3
2,Western,2


## We can mix and match SQL and dot syntax

In [17]:
regions3 = spark.sql('select regionname, regionid from regions where regionid < 4 order by regionname')
regions3 = spark.sql('select regionname, regionid from regions where regionid < 4').sort('regionname')
regions3 = spark.sql('select regionname, regionid from regions').filter('regionid < 4').sort('regionname')

display(regions3)

,regionname,regionid
0,Eastern,1
1,Northern,3
2,Western,2


## We could also read file directly without the need to have them pre-defined in Hive first. This makes things more flexible and allows for reading more file types and for dynamic schema resolution.

## Read in a file to a Spark DataFrame.

In [18]:
territories = spark.read.csv('file:///class/datasets/northwind/CSVHeaders/territories'
                             , inferSchema = True, header=True)
print(territories)
display(territories)


DataFrame[TerritoryID: int, TerritoryName: string, RegionID: int]


,TerritoryID,TerritoryName,RegionID
0,1581,Westboro,1
1,1730,Bedford,1
2,1833,Georgetow,1
3,2116,Boston,1
4,2139,Cambridge,1
5,2184,Braintree,1
6,2903,Providence,1
7,3049,Hollis,3
8,3801,Portsmouth,3
9,6897,Wilton,1


## But if we want we can define a specific schema and use it instead of dynamic resolution

In [19]:
region_schema = StructType([
    StructField('regionid', IntegerType()), 
    StructField('regionname', StringType())
])

regions4 = spark.read.csv('hdfs://localhost:9000/regions', schema = region_schema, header = False, sep = ',')

display(regions4)


,regionid,regionname
0,1,Eastern
1,2,Western
2,3,Northern
3,4,Southern


### Really the SQL is being converted to something like the following Spark commands.

In [22]:
%%sparksql regions5
SELECT regionname, regionid FROM regions WHERE regionid < 4 ORDER BY regionid


capture dataframe to local variable `regions5`


regionname,regionid
Eastern,1
Western,2
Northern,3


In [23]:
regionschema = StructType([
    StructField('regionid', IntegerType()), 
    StructField('regionname', StringType())
])

regions6 = (spark.read.csv('hdfs://localhost:9000/regions', header=False
                         , schema = regionschema, sep = ',')
                .select('regionname', 'regionid')
                .where('regionid < 4')
                .sort('regionid'))
display(regions6)

,regionname,regionid
0,Eastern,1
1,Western,2
2,Northern,3


## Use createOrReplaceTempView to create a virtual table in the Hive catalog and then it can be queried using SQL as if it were a hive table.

In [25]:
# this is the territories we defined a few blocks up, not the one in the Hive catalog
display(territories)

,TerritoryID,TerritoryName,RegionID
0,1581,Westboro,1
1,1730,Bedford,1
2,1833,Georgetow,1
3,2116,Boston,1
4,2139,Cambridge,1
5,2184,Braintree,1
6,2903,Providence,1
7,3049,Hollis,3
8,3801,Portsmouth,3
9,6897,Wilton,1


In [28]:
territories.createOrReplaceTempView('territories')
t1 = spark.sql('SELECT regionid, territoryid, territoryname FROM territories WHERE regionid = 1')
display(t1)
print(t1.count())

display(territories.select('regionid', 'territoryid', 'territoryname').filter('regionid=1'))

,regionid,territoryid,territoryname
0,1,1581,Westboro
1,1,1730,Bedford
2,1,1833,Georgetow
3,1,2116,Boston
4,1,2139,Cambridge
5,1,2184,Braintree
6,1,2903,Providence
7,1,6897,Wilton
8,1,7960,Morristown
9,1,8837,Edison


19


,regionid,territoryid,territoryname
0,1,1581,Westboro
1,1,1730,Bedford
2,1,1833,Georgetow
3,1,2116,Boston
4,1,2139,Cambridge
5,1,2184,Braintree
6,1,2903,Providence
7,1,6897,Wilton
8,1,7960,Morristown
9,1,8837,Edison


## This is the regions table from Hive and the territories DataFrame we locally defined

In [29]:
%%sparksql
select r.regionid, r.regionname, t.territoryid, t.territoryname
from regions as r
join territories as t on r.regionid = t.regionid
order by r.regionid, t.territoryid


only showing top 20 row(s)


regionid,regionname,territoryid,territoryname
1,Eastern,1581,Westboro
1,Eastern,1730,Bedford
1,Eastern,1833,Georgetow
1,Eastern,2116,Boston
1,Eastern,2139,Cambridge
1,Eastern,2184,Braintree
1,Eastern,2903,Providence
1,Eastern,6897,Wilton
1,Eastern,7960,Morristown
1,Eastern,8837,Edison


## Spark DataFrames can be saved to a Hive table using either the saveAsTable method or writing a SQL query that uses CREATE TABLE AS.

In [34]:
help(territories.write.saveAsTable)

Help on method saveAsTable in module pyspark.sql.readwriter:

saveAsTable(name, format=None, mode=None, partitionBy=None, **options) method of pyspark.sql.readwriter.DataFrameWriter instance
    Saves the content of the :class:`DataFrame` as the specified table.
    
    In the case the table already exists, behavior of this function depends on the
    save mode, specified by the `mode` function (default to throwing an exception).
    When `mode` is `Overwrite`, the schema of the :class:`DataFrame` does not need to be
    the same as that of the existing table.
    
    * `append`: Append contents of this :class:`DataFrame` to existing data.
    * `overwrite`: Overwrite existing data.
    * `error` or `errorifexists`: Throw an exception if data already exists.
    * `ignore`: Silently ignore this operation if data already exists.
    
    .. versionadded:: 1.4.0
    
    Notes
    -----
    When `mode` is `Append`, if there is an existing table, we will use the format and
    options o

In [36]:
territories.write.mode('overwrite').format('CSV').saveAsTable('territories3')
#territories.write.saveAsTable('territories2', mode='overwrite')
#territories.write.saveAsTable('territories3', mode='overwrite', format='csv')



AnalysisException: Can not create the managed table('`territories3`'). The associated location('hdfs://localhost:9000/user/hive/warehouse/territories3') already exists.

In [40]:
spark.sql("create table territories4 STORED AS PARQUET LOCATION '/territories4' as select * from territories")


DataFrame[]

## Queries use standard HQL to mix Hive tables and virtual tables. Both are read into a Spark DataFrame and the processing happens at the Spark level, not at the Hive level. HQL is just used to parse the logic into the corresponding Spark methods.

In [ ]:
query = """
select r.regionid, r.regionname, t.territoryid, t.territoryname 
from regions as r 
join territories as t on r.regionid = t.regionid 
order by r.regionid, t.territoryid
"""
rt = spark.sql(query)
rt.show(10)

tr = regions.join(territories, regions.regionid == territories.RegionID). \
     select('regions.regionid', 'regionname', 'TerritoryID', 'TerritoryName') .\
    orderBy('regionid', 'territoryid')
tr.show(10)


## LAB: ## 
### Read the northwind JSON products and make it into a TempView and do the same with the CSVHeaders version of categories.
<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Look at Day 2 or below to copy the code to load in the DataFrames
<br>
Turn each DataFrame into a temporary view
<br>
<br>
</p>
</details>

<details><summary>Click for <b>code</b></summary>
<p>

```python
categories = spark.read.load('file:///class/datasets/northwind/CSVHeaders/categories', format = 'csv', sep = ',', inferSchema = True, header = True)

prodSchema = StructType([
    StructField('productid', IntegerType()), 
    StructField('productname', StringType()),
    StructField('supplierid', IntegerType()), 
    StructField('categoryid', IntegerType()), 
    StructField('quantityperunit', StringType()), 
    StructField('unitprice', FloatType()), 
    StructField('unitsinstock', IntegerType()), 
    StructField('unitsonorder', IntegerType()), 
    StructField('reorderlevel', IntegerType()), 
    StructField('discontinued', IntegerType())
])
products = spark.read.json('file:///class/datasets/northwind/JSON/products', schema=prodSchema)

categories.createOrReplaceTempView('categories')
products.createOrReplaceTempView('products')```
</p>
</details>

In [ ]:
%%sparksql
select * from products as p
join categories as c on p.categoryid = c.categoryid


## Install the MySQL Python connector. This has nothing to do with Spark, but if you want to run SQL queries directly it is helpful.
It's already on our machines so we don't need to run this now.

In [ ]:
! pip install mysql-connector-python

## Let's make sure we have a database for northwind and no regions table.

In [ ]:
import mysql.connector
try:
    cn = mysql.connector.connect(host='localhost', user='test', password='password')
    cursor = cn.cursor()
    cursor.execute('create database if not exists northwind')
    cn.close()

    cn = mysql.connector.connect(host='localhost', user='test', password='password', database='northwind')
    cursor = cn.cursor()    
    cursor.execute('drop table if exists regions')
    cn.close()
except:
    print('something went wrong')
else:
    print('success')



## Write a DataFrame to a SQL database.

In [ ]:
regions.show()

In [ ]:
regions.write.format("jdbc").options(url="jdbc:mysql://localhost/northwind", \
                                     driver='com.mysql.jdbc.Driver', \
                                     dbtable='regions', \
                                     user='test', password = "password", mode = "append", \
                                     useSSL = "false").save()
print('Done')

## Read a SQL table into a Spark DataFrame.

In [ ]:
regions2 = spark.read.format("jdbc").options(url="jdbc:mysql://localhost/northwind", \
                                             driver="com.mysql.jdbc.Driver", \
                                             dbtable= "regions", \
                                             user="test", password="password").load()
regions2.show()


## If you don't wont to bring an entire SQL table into a spark DataFrame use query instead of dbtable.

In [ ]:
regions3 = spark.read.format("jdbc").options(url="jdbc:mysql://localhost/northwind", \
                                             driver="com.mysql.jdbc.Driver", \
                                             query= "select regionid, regionname from regions where regionid < 3", \
                                             user="test", password="password").load()
regions3.show()



## Creating the regions2 DataFrame does not execute anything yet, but by making the DataFrame into a Temp View then running a Spark SQL query, it tells Spark to read the SQL data into a DataFrame and then use the cluster to do the processing, not the SQL source.

In [ ]:
regions2.createOrReplaceTempView('regions2')
spark.sql('select * from regions2 where regionid < 3').show()

In [ ]:
spark.read.table('regions2').where('regionid < 3').show()

## Alternate ways to code a query using SQL and methods.

In [ ]:
x = spark.sql('select count(*) from northwind.regions').collect()
print(x)
print(x[0], x[0]['count(1)'])

x = spark.sql('select count(*) as cnt from northwind.regions').collect()
print(x)
print(x[0], x[0].cnt)

print(spark.sql('select * from northwind.regions').count())
print(spark.read.table('northwind.regions').count())

print(spark.sql('select * from northwind.regions where regionid < 4').count())
print(spark.read.table('northwind.regions').where('regionid<4').count())

## Using SQL you can use familiar syntax instead of withColumn or withColumnRenamed methods.
Note the expr function needs to be imported when you want to use a stringified SQL function using dot syntax.

In [ ]:
from pyspark.sql.functions import expr

t1 = spark.sql('select TerritoryID as TerrID, UPPER(TerritoryName) as TerritoryName, RegionID from northwind.territories')
t1.show(5)

from pyspark.sql.functions import expr
territories.withColumn('TerritoryName', expr('UPPER(TerritoryName)')).withColumnRenamed('TerritoryID', 'TerrID').show(5)


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# This won't work though if you want to use python functions, you need to go another step
# territories.withColumn('TerritoryName', territories.TerritoryName.upper()).show()

# You need to make the python function callable by spark by wrapping it in the udf function
# which tells spark what datatype it returns

def joeyfunc(x: str) -> str:
    return x[::-1]

#territories.withColumn('TerritoryName', udf(str.title, StringType())(territories.TerritoryName)).show()
territories.withColumn('TerritoryName', udf(joeyfunc, StringType())(territories.TerritoryName)).show()



## If you want to use a function that is not a standard Python or SQL function, you can always create one in Python and make it callable from Spark.

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def inventoryvalue(quantity: int, price: float) -> float:
    return quantity * price

def inventoryvalue(quantity, price):
    return quantity * price

# Turn the Python function into a Spark callable function
invvalue = udf(inventoryvalue, FloatType())
p = products
p2 = p.withColumn('value', invvalue(p.unitsinstock, p.unitprice))
display(p2)




## Python decorators are an even better option.

In [ ]:
@udf(FloatType())
def inventoryvalue(quantity, price):
    return quantity * price

p2 = p.withColumn('value', inventoryvalue(p.unitsinstock, p.unitprice))
display(p2)


In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

def inventoryvalue(quantity, price):
    return quantity * price

# Or dynamically wrap it, but it's harder to read
p2 = p.withColumn('value', udf(inventoryvalue, FloatType())(p.unitsinstock, p.unitprice))
display(p2)



## To make it easier though, you could make the Python function into a udf that SQL can understand similar to how you can make a DataFrame seem like a virtual table with createOrReplaceTempView.

In [ ]:
def reverseString(x):
    return x[::-1]

spark.udf.register('reverse', reverseString, StringType())

spark.sql('select *, reverse(TerritoryName) as Reversed from Territories').orderBy('Reversed').show()

## HQL has collect_set and collect_list functions to aggregate items into a list instead of summing them up. 

In [ ]:
from pyspark.sql.functions import collect_list
territories.groupBy(territories.RegionID).agg(collect_list(territories.TerritoryName)).show()

tr1 = spark.sql("SELECT RegionID, collect_list(TerritoryName) AS TerritoryList FROM Territories GROUP BY RegionID")
tr1.show()
tr1.printSchema()
print(tr1.take(1))


## Instead of a simple datatype, you could also collect complex structured objects using the HQL NAMED_STRUCT.

In [ ]:
sql = """
SELECT r.RegionID, r.RegionName
, COLLECT_SET(NAMED_STRUCT("TerritoryID", TerritoryID, "TerritoryName", TerritoryName)) AS TerritoryList
FROM northwind.Regions AS r
JOIN Territories AS t ON r.RegionID = t.RegionID
GROUP BY r.RegionID, r.RegionName
ORDER BY r.RegionID
"""

tr2 = spark.sql(sql)
tr2.printSchema()
print(tr2)
tr2.show()
print(tr2.take(2))
tr2.write.json('TerritoryRegion.json')
spark.sql('create table TerritoryRegion as ' + sql)

## If you have data that is already collected into a complex datatype and want to flatten it, you could use HQL EXPLODE function.

## You could use the Spark explode method.

In [ ]:
from pyspark.sql.functions import explode
tr1.select('RegionID', explode('TerritoryList')).withColumnRenamed('col','TerritoryName').show()


## Or if the DataFrame is turned into a Temp View, you could use the HQL query to do it.

In [ ]:
tr1.createOrReplaceTempView('RegionTerritories')
sql = """
SELECT RegionID, TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS TerritoryName
ORDER BY RegionID, TerritoryName
"""
spark.sql(sql).show()

## Or you could select specific elements from a collection.

In [ ]:
tr2.createOrReplaceTempView('RegionTerritories')
spark.sql("select RegionId, RegionName, TerritoryList[0] as First, TerritoryList[size(TerritoryList) - 1] as Last, size(TerritoryList) as TerritoryCount from RegionTerritories").show()


## If the array is of structs, note the syntax of fetching the elements from the struct uses the . like an object property.

In [ ]:
sql = """
SELECT RegionID, RegionName, Territory.TerritoryID AS TerritoryID
, Territory.TerritoryName AS TerritoryName
FROM RegionTerritories
LATERAL VIEW EXPLODE(TerritoryList) EXPLODED_TABLE AS Territory
"""
spark.sql(sql).show()


## HOMEWORK: ## 
**First Challenge**

Create a Python function to determine if a number is odd or even and use that to select only the even numbered shippers from the TSV folder of northwind. Note the TSV file does not have headers so you will need to do something to make the DataFrame have a meaningful structure. I would suggest using Spark SQL as much as possible to rename and cast the columns which are ShipperID, CompanyName, and Phone.

**Second Challenge**

Take the Order_LineItems.json folder, read it into a DataFrame, and flatten it and then calculate the average price paid for a product.

<br>
<details><summary>Click for <b>hint</b></summary>
<p>
Take a look at the MakeOrders_LineItems.py file provided to see how the Order_LineItems.json was generated in the first place
<br>
Use modulus with remainder of zero to determine if something is even
<br>
Use udf to make a version of the function that is callable using dot syntax and udf.register to make a version callable from within a SQL string
<br>
Use LATERAL VIEW EXPLODE() EXPLODED_TABLE to flatten out the nested format file
<br>
Once flattened do a traditional aggregate to calculate the average
<br>
<br>
</p>
</details>





In [ ]:
%load_ext sparksql_magic

In [ ]:
%%sparksql
select * from regions